# Importar Bibliotecas Necessárias

In [1]:
# Importar as bibliotecas

# -*- coding: utf-8 -*-

import numpy as np
from timeit import default_timer as timer
import random
import serial
import serial.tools.list_ports
import os
from math import sqrt
import argparse
import matplotlib as plt
import pandas as pd
import tensorflow as tf

from datetime import datetime
from scipy.fft import fft

# https://www.linkedin.com/pulse/lendo-arquivos-csv-com-pandas-rog%C3%A9rio-guimar%C3%A3es-de-campos-j%C3%BAnior/
# https://stackabuse.com/python-list-files-in-a-directory/

C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ander\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

# Rede LSTM

In [2]:
# Carregar dados MPL para modelo RNN

# Definição do paciente e os diretórios
nome = "Rafaela_Teste"

# Caminho dos dados
filename = f"Dataset\\{nome}\\dados_MLP.csv"

# Definir lables dos dados
names = ['Part', 'Tipo_Movimento']

# Colocar cada Dataframe dentro de um dicionário
df = pd.read_csv(f"{filename}", sep=";", header=None, names=names, index_col=None)

display(df.head(5))
display(df.shape)

,Part,Tipo_Movimento
0,"1.58,1.6,1.59,1.59,1.57,1.57,1.54,1.54,1.53,1....",1
1,"1.58,1.6,1.59,1.59,1.57,1.57,1.54,1.54,1.53,1....",1
2,"1.58,1.6,1.59,1.59,1.57,1.57,1.54,1.54,1.53,1....",1
3,"1.69,1.84,1.93,1.95,1.94,1.87,1.75,1.61,1.51,1...",1
4,"1.69,1.84,1.93,1.95,1.94,1.87,1.75,1.61,1.51,1...",1


(10, 2)

# Extração de características do sinal (Input Layer)

In [3]:
# Transforma de str para float

def str2float(Part):
    for i in range(0, len(Part)):
        Part[i] = Part[i].split(',')
        for j in range(0, len(Part[i])):
            Part[i][j] = float(Part[i][j]) - 1.65 # "-1.65" -> Regularizar os dados no zero (Somente para Rafaela Teste - DADOS ANTIGOS)

    return Part

In [4]:
# Calcula: Comprimento do traço do sinal

def waveform_length(dado, segments):
    # segment[sample] = Xk
    # segment[sample-1] = Xk-1
    
    l0_list = []
    
    segment = int(len(dado)/segments)
    aux = 0
    aux_2 = segment
    
    for s in range(0, segments):
        l0 = 0
        aux_2 = segment * (s+1)
        for sample in range(aux, aux_2):
            l0 = l0 + abs(dado[sample] - dado[sample-1])
        
        l0_list.append(l0)
        aux = aux_2
        
    # print (f"l0: {l0_list}")
    return l0_list

In [5]:
# Calcula: Média do Valor Absoluta

# Recebe uma amostra com 400 pontos e faz a media absoluta baseada na quantidade de segmentos
def mean_absolute_value(dado, segments):
    MAV = []
    
    segment = int(len(dado)/segments)
    aux = 0
    aux_2 = segment
    
    
    for s in range(0, segments): 
        soma = 0
        aux_2 = segment * (s+1)
        for sample in range(aux, aux_2):
            soma = soma + abs(dado[sample])
    
        MAV.append(soma/segment)
        aux = aux_2 - 1
        
    # print (f"MAV: {MAV}")
    return MAV

In [6]:
# Calcula: Zero Crossing (ZC)

def zero_crossing(dado, segments): 
    ZC = []

    segment = int(len(dado)/segments)
    aux = 0
    aux_2 = segment
    
    # X0 = Xk
    # X1 = Xk+1
    
    for s in range(0, segments): 
        count = 0
        aux_2 = segment * (s+1)
        for sample in range(aux, aux_2-1):
            X0 = dado[sample]
            X1 = dado[sample + 1]
            # abs(Xk - Xk-1) >= 10mV
            # limiar (threshhold) = 10mV. Noise INA332 = 7uVp-p. Para ter contagem de ZC falsos
            if (((X0 > 0 and X1 < 0) or (X0 < 0 and X1 > 0)) and (abs(X0 - X1) >= 0.01)):
                count += 1
                # print (f"Diferença para ZC: {abs(X0 - X1)}")
        
        ZC.append(count)
        aux = aux_2 - 1
    
    # print (f"ZC: {ZC}")    
    return ZC
    
    

In [7]:
# Calcula: A mudança de sinais

def slope_sign_changes(dado, segments):
    SSC = []
    
    # X0 = Xk-1
    # X1 = Xk
    # X2 = Xk+1
    
    segment = int(len(dado)/segments)
    aux = 1
    aux_2 = segment
    
    for s in range(0, segments): 
        count = 0
        aux_2 = segment * (s+1)
        for sample in range(aux, aux_2 - 1):
            X0 = dado[sample - 1]
            X1 = dado[sample]
            X2 = dado[sample + 1]
            if (((X1 > X0  and X1 > X2) or (X1 < X0 and X1 < X2)) and ((abs(X1 - X2) >= 0.01) or (abs(X1 - X0) >= 0.01))):
                count += 1
                
        SSC.append(count)
        aux = aux_2 - 1
    
    # print (f"SSC: {SSC}")
    return SSC

In [8]:
# Calcula: Ângulo de inclinação da média do valor absoluto

def mean_absolute_value_slope(dado, segments):
    # MAV = Xi
    # MAV1 = Xi+1
    
    MAVSLP = []
    MAV = mean_absolute_value(dado, segments)
    
    for s in range(0, (segments-1)):
        MAVSLP.append(abs(MAV[s+1] - MAV[s])) 
    
    MAVSLP.append(0.0)
    
    # print (f"MAVSLP{MAVSLP}")        
    return MAVSLP

In [9]:
# Calcula: RMS 

def f_rms(dado, segments):
    Vrms = []
    
    segment = int(len(dado)/segments)
    aux = 0
    aux_2 = segment
    
    for s in range(0, segments): 
        soma = 0
        aux_2 = segment * (s+1)
        for sample in range(aux, aux_2):
            soma = soma + (dado[sample] ** 2) 

        Vrms.append(sqrt(soma/segment))
        aux = aux_2 - 1

    return Vrms

In [10]:
# Extrai os parâmetros necessários
# FOI ORGANIZADO DE UMA FORMA QUE FOSSE FORMADO UMA AMOSTRA POR ARRAY 

# Adendo: Basicamente será testada a rede de Hudgins, mas diferente do artigo original será usada uma rede melhorada da RNN,
# que neste caso é a LSTM

# Transformar em um array (lista de listas)
Part = str2float(df.Part.tolist())

# Criação do DataFrame para LSTM
segments = 5 # Assim como Hudgins é possível modificar a quantidade de segmentos e ver o quanto melhora ou piora a acurácia obtida
line = list()

# Extrair características
for p in Part:
    line.append(mean_absolute_value(p, segments))
    line.append(waveform_length(p, segments))
    line.append(f_rms(p, segments))
    line.append(mean_absolute_value_slope(p, segments))
    line.append(zero_crossing(p, segments))
    line.append(slope_sign_changes(p, segments))
    
# Separação em X e y    
X = np.array(line)
y = np.array(df.Tipo_Movimento.tolist())
y = np.reshape(y, (len(Part),1))

# Reshape deve ser: (quantidade de amostras,quantidade de features,segmentos da serie temporal)
X = np.reshape(X, (len(Part),6,segments))
display(X)



array([[[2.56375000e-01, 1.87000000e-01, 2.38750000e-01, 1.66750000e-01,
         2.14625000e-01],
        [3.33000000e+00, 2.93000000e+00, 4.79000000e+00, 2.76000000e+00,
         4.19000000e+00],
        [3.01825695e-01, 2.32481182e-01, 2.77889366e-01, 1.83521116e-01,
         2.74278599e-01],
        [6.93750000e-02, 5.17500000e-02, 7.20000000e-02, 4.78750000e-02,
         0.00000000e+00],
        [1.00000000e+00, 2.00000000e+00, 7.00000000e+00, 4.00000000e+00,
         3.00000000e+00],
        [1.10000000e+01, 1.40000000e+01, 9.00000000e+00, 1.30000000e+01,
         1.00000000e+01]],

       [[2.56375000e-01, 1.87000000e-01, 2.38750000e-01, 1.66750000e-01,
         2.14625000e-01],
        [3.33000000e+00, 2.93000000e+00, 4.79000000e+00, 2.76000000e+00,
         4.19000000e+00],
        [3.01825695e-01, 2.32481182e-01, 2.77889366e-01, 1.83521116e-01,
         2.74278599e-01],
        [6.93750000e-02, 5.17500000e-02, 7.20000000e-02, 4.78750000e-02,
         0.00000000e+00],
        

In [13]:
# Extrai os parâmetros necessários de uma segunda forma
# FOI ORGANIZADO DE UMA FORMA QUE FOSSE FORMADO UMA FEATURE POR ARRAY 

# Adendo: Basicamente será testada a rede de Hudgins, mas diferente do artigo original será usada uma rede melhorada da RNN,
# que neste caso é a LSTM

# Criação do Array para LSTM
segments = 5 # Assim como Hudgins é possível modificar a quantidade de segmentos e ver o quanto melhora ou piora a acurácia obtida
MAV = list()
WL = list()
RMS = list()
MAVSL = list()
ZC = list()
SSC = list()

# Extrair características
for p in Part:
    MAV.append(mean_absolute_value(p, segments))
    WL.append(waveform_length(p, segments))
    RMS.append(f_rms(p, segments))
    MAVSL.append(mean_absolute_value_slope(p, segments))
    ZC.append(zero_crossing(p, segments))
    SSC.append(slope_sign_changes(p, segments))
    
# Separação em X e y    

# Reshape deve ser: (quantidade de features, quantidade de amostras,segmentos da serie temporal)
X = np.array([MAV,WL,RMS,MAVSL,ZC,SSC])
X = np.reshape(X, (6,len(Part),segments))

# Reshape deve ser: (quantidade de amostras, 1)
y = np.array(df.Tipo_Movimento.tolist())
y = np.reshape(y, (len(Part),1))


display(X)

array([[[2.56375000e-01, 1.87000000e-01, 2.38750000e-01, 1.66750000e-01,
         2.14625000e-01],
        [2.56375000e-01, 1.87000000e-01, 2.38750000e-01, 1.66750000e-01,
         2.14625000e-01],
        [2.56375000e-01, 1.87000000e-01, 2.38750000e-01, 1.66750000e-01,
         2.14625000e-01],
        [2.31750000e-01, 1.97000000e-01, 1.77750000e-01, 1.91125000e-01,
         1.72750000e-01],
        [2.31750000e-01, 1.97000000e-01, 1.77750000e-01, 1.91125000e-01,
         1.72750000e-01],
        [2.31750000e-01, 1.97000000e-01, 1.77750000e-01, 1.91125000e-01,
         1.72750000e-01],
        [2.31750000e-01, 1.97000000e-01, 1.77750000e-01, 1.91125000e-01,
         1.72750000e-01],
        [2.31750000e-01, 1.97000000e-01, 1.77750000e-01, 1.91125000e-01,
         1.72750000e-01],
        [2.31750000e-01, 1.97000000e-01, 1.77750000e-01, 1.91125000e-01,
         1.72750000e-01],
        [2.31750000e-01, 1.97000000e-01, 1.77750000e-01, 1.91125000e-01,
         1.72750000e-01]],

       [

In [260]:
# PREPARAÇÃO DOS DADOS DE ACORDO COM O EXEMPLO: https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
# ESTÁ CORRETO?

# prepare data for lstm
from pandas import concat
from sklearn.preprocessing import MinMaxScaler

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    #; forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# load dataset
values = X[1,:,:]
# frame as supervised learning
reframed = series_to_supervised(values, 1, 1)
print(reframed.head())

   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)    var1(t)  \
1   0.256375   0.187000   0.238750   0.166750   0.214625   3.330000   
2   3.330000   2.930000   4.790000   2.760000   4.190000   0.301826   
3   0.301826   0.232481   0.277889   0.183521   0.274279   0.069375   
4   0.069375   0.051750   0.072000   0.047875   0.000000   1.000000   
5   1.000000   2.000000   7.000000   4.000000   3.000000  11.000000   

     var2(t)   var3(t)    var4(t)    var5(t)  
1   2.930000  4.790000   2.760000   4.190000  
2   0.232481  0.277889   0.183521   0.274279  
3   0.051750  0.072000   0.047875   0.000000  
4   2.000000  7.000000   4.000000   3.000000  
5  14.000000  9.000000  13.000000  10.000000  


In [261]:
# PREPARAÇÃO DOS DADOS DE ACORDO COM O EXEMPLO: https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
# ESTÁ CORRETO?

# split into train and test sets
values = reframed.values
n_train_hours = 2
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(2, 1, 9) (2,) (3, 1, 9) (3,)


In [218]:
# TENTATIVA DE TREINO E TESTE ANTIGO (REFORMULAR)
from sklearn.model_selection import train_test_split

display(X.shape)
display(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

display(X)
display(y)
    
# https://gusrabbit.com/intuition/treino-teste/

(10, 6, 5)

(10, 1)

array([[[2.56375000e-01, 1.87000000e-01, 2.38750000e-01, 1.66750000e-01,
         2.14625000e-01],
        [3.33000000e+00, 2.93000000e+00, 4.79000000e+00, 2.76000000e+00,
         4.19000000e+00],
        [3.01825695e-01, 2.32481182e-01, 2.77889366e-01, 1.83521116e-01,
         2.74278599e-01],
        [6.93750000e-02, 5.17500000e-02, 7.20000000e-02, 4.78750000e-02,
         0.00000000e+00],
        [1.00000000e+00, 2.00000000e+00, 7.00000000e+00, 4.00000000e+00,
         3.00000000e+00],
        [1.10000000e+01, 1.40000000e+01, 9.00000000e+00, 1.30000000e+01,
         1.00000000e+01]],

       [[2.56375000e-01, 1.87000000e-01, 2.38750000e-01, 1.66750000e-01,
         2.14625000e-01],
        [3.33000000e+00, 2.93000000e+00, 4.79000000e+00, 2.76000000e+00,
         4.19000000e+00],
        [3.01825695e-01, 2.32481182e-01, 2.77889366e-01, 1.83521116e-01,
         2.74278599e-01],
        [6.93750000e-02, 5.17500000e-02, 7.20000000e-02, 4.78750000e-02,
         0.00000000e+00],
        

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [8]:
# Criar a rede LSTM

In [ ]:
# Criar a rede GRU

In [10]:
# Verificar acurácia e perda